# Fast String/Matrix Operations

Benchmarks and examples for `dev/strings.py` - high-performance string generation using uint8 byte matrices.

In [3]:
import numpy as np
import timeit

from strings import (
    # Conversion utilities
    strs2u8mat, u8m2S, u8m2s, u82S, u82s, s2u8, sep, make_u8mat,
    # Date reading
    read_1digit, read_2digits, read_4digits,
    get_uint8_ym, get_uint8_ymd,
    # Date writing
    make_ym, add_months_ym, add_months_ym_inplace, 
    add_months2specs_inplace,add_months2specs_inplace_NF,
    # Calendar functions
    make_ym_matrix, make_ymd_matrix,
    make_calendar_from_ranges, make_calendar_from_ranges_par,
    make_calendar_from_specs_par,
    # Cartesian product
    cartesian_product, cartesian_product_np, cartesian_product_par,
    # Unfurl functions
    unfurl, unfurl_by_spec, unfurl_by_spec_sep,
    unfurl_concat, unfurl_concat_sep,
)

def benchmark(func, data, n_runs=5):
    """Run benchmark and return best time in milliseconds."""
    times = timeit.repeat(lambda: func(data), number=1, repeat=n_runs)
    return min(times) * 1000

def run_bench(func, n_runs=5):
    """Run benchmark without data argument."""
    times = timeit.repeat(func, number=1, repeat=n_runs)
    return min(times) * 1000

## Calendar Generation

Generate date strings from year-month ranges.

In [2]:
# Large scale test: 799 straddles × 25 years × 12 months = 239,700 month-ranges
src = np.array(
    [[y, m+1, y, m+1] for i in range(1, 800) for y in range(2001, 2026) for m in range(0, 12)],
    dtype=np.int64
)

src_idx, cal = make_calendar_from_ranges(src)

print(f"Input ranges: {src.shape[0]:,}")
print(f"Output rows: {cal.shape[0]:,}")
print(f"Output cols: {cal.shape[1]}")
print(f"Unique source ids: {np.unique(src_idx).shape[0]:,}")

Input ranges: 239,700
Output rows: 7,295,669
Output cols: 10
Unique source ids: 239,700


In [3]:
# Sample output
print("Sample dates (rows 55-65):")
for s in u8m2S(cal)[55:65].astype(f"U{cal.shape[1]}"):
    print(f"  {s}")

print("\nLast 3 dates:")
for s in u8m2S(cal)[-3:].astype(f"U{cal.shape[1]}"):
    print(f"  {s}")

Sample dates (rows 55-65):
  2001-02-25
  2001-02-26
  2001-02-27
  2001-02-28
  2001-03-01
  2001-03-02
  2001-03-03
  2001-03-04
  2001-03-05
  2001-03-06

Last 3 dates:
  2025-12-29
  2025-12-30
  2025-12-31


In [4]:
# Benchmark
t = benchmark(make_calendar_from_ranges, src)
print(f"make_calendar_from_ranges: {t:.2f} ms")
print(f"Throughput: {cal.shape[0]/t/1000:.1f}M rows/sec")

make_calendar_from_ranges: 12.38 ms
Throughput: 589.5M rows/sec


In [5]:
# Parallel calendar generation benchmark
_ = make_calendar_from_ranges_par(src)  # Warm up

t_seq = benchmark(make_calendar_from_ranges, src)
t_par = benchmark(make_calendar_from_ranges_par, src)

print(f"make_calendar_from_ranges:     {t_seq:6.2f} ms")
print(f"make_calendar_from_ranges_par: {t_par:6.2f} ms")
print(f"Speedup: {t_seq/t_par:.2f}x")

make_calendar_from_ranges:      12.43 ms
make_calendar_from_ranges_par:   2.42 ms
Speedup: 5.15x


In [6]:
# Scaling comparison for calendar generation
print("Sequential vs Parallel calendar generation:")
print(f"{'Ranges':>12} {'Days':>12} {'seq (ms)':>10} {'par (ms)':>10} {'speedup':>8}")
print("-" * 56)

for n_straddles in [100, 500, 800]:
    test_src = np.array(
        [[y, m+1, y, m+1] for i in range(n_straddles) for y in range(2001, 2026) for m in range(12)],
        dtype=np.int64
    )
    
    # Warm up
    _ = make_calendar_from_ranges(test_src)
    _ = make_calendar_from_ranges_par(test_src)
    
    t_seq = benchmark(make_calendar_from_ranges, test_src)
    t_par = benchmark(make_calendar_from_ranges_par, test_src)
    
    _, test_cal = make_calendar_from_ranges(test_src)
    n_days = test_cal.shape[0]
    speedup = t_seq / t_par if t_par > 0 else float('inf')
    
    print(f"{test_src.shape[0]:>12,} {n_days:>12,} {t_seq:>10.2f} {t_par:>10.2f} {speedup:>7.2f}x")

Sequential vs Parallel calendar generation:
      Ranges         Days   seq (ms)   par (ms)  speedup
--------------------------------------------------------
      30,000      913,100       2.04       0.79    2.57x
     150,000    4,565,500       7.81       1.54    5.08x
     240,000    7,304,800      15.29       3.11    4.92x


### Calendar from Specs

Generate dates directly from straddle spec strings (`|YYYY-MM|YYYY-MM|...`).

In [7]:
# Create straddle specs as uint8 matrix
# Format: |YYYY-MM|YYYY-MM|extra...
specs_list = [f"|{y}-{m:02d}|{y}-{m:02d}|extra" for i in range(1, 800) for y in range(2001, 2026) for m in range(1, 13)]
specs = strs2u8mat(specs_list)

print(f"Specs shape: {specs.shape}")
print(f"Sample specs:")
for s in u8m2S(specs[:3]).astype(f"U{specs.shape[1]}"):
    print(f"  {s}")

Specs shape: (239700, 22)
Sample specs:
  |2001-01|2001-01|extra
  |2001-02|2001-02|extra
  |2001-03|2001-03|extra


In [8]:
# Run and verify output
src_idx_spec, cal_spec = make_calendar_from_specs_par(specs)

print(f"Input specs: {specs.shape[0]:,}")
print(f"Output rows: {cal_spec.shape[0]:,}")
print(f"Output cols: {cal_spec.shape[1]}")

print(f"\nSample output dates:")
for s in u8m2S(cal_spec[55:65]).astype(f"U{cal_spec.shape[1]}"):
    print(f"  {s}")

Input specs: 239,700
Output rows: 7,295,669
Output cols: 10

Sample output dates:
  2001-02-25
  2001-02-26
  2001-02-27
  2001-02-28
  2001-03-01
  2001-03-02
  2001-03-03
  2001-03-04
  2001-03-05
  2001-03-06


In [9]:
# Benchmark: specs vs ranges
# Warm up
_ = make_calendar_from_specs_par(specs)
_ = make_calendar_from_ranges_par(src)

t_specs = benchmark(make_calendar_from_specs_par, specs)
t_ranges = benchmark(make_calendar_from_ranges_par, src)

print(f"make_calendar_from_specs_par:  {t_specs:6.2f} ms")
print(f"make_calendar_from_ranges_par: {t_ranges:6.2f} ms")
print(f"Overhead for parsing specs: {(t_specs - t_ranges) / t_ranges * 100:.1f}%")
print(f"\nThroughput (specs): {cal_spec.shape[0]/t_specs/1000:.1f}M rows/sec")

make_calendar_from_specs_par:    3.68 ms
make_calendar_from_ranges_par:   3.18 ms
Overhead for parsing specs: 15.9%

Throughput (specs): 1980.5M rows/sec


In [10]:
# Scaling comparison for specs
print("Specs calendar generation scaling:")
print(f"{'Specs':>12} {'Days':>12} {'time (ms)':>10} {'rows/sec':>12}")
print("-" * 50)

for n_straddles in [100, 500, 800]:
    test_specs = strs2u8mat([
        f"|{y}-{m:02d}|{y}-{m:02d}|" 
        for i in range(n_straddles) 
        for y in range(2001, 2026) 
        for m in range(1, 13)
    ])
    
    # Warm up
    _ = make_calendar_from_specs_par(test_specs)
    
    t = benchmark(make_calendar_from_specs_par, test_specs)
    _, test_cal = make_calendar_from_specs_par(test_specs)
    n_days = test_cal.shape[0]
    
    print(f"{test_specs.shape[0]:>12,} {n_days:>12,} {t:>10.2f} {n_days/t/1000:>11.1f}M")

Specs calendar generation scaling:
       Specs         Days  time (ms)     rows/sec
--------------------------------------------------
      30,000      913,100       0.67      1366.4M
     150,000    4,565,500       3.43      1332.9M
     240,000    7,304,800       3.75      1947.3M


## Cartesian Product

N-way cartesian product of byte matrices.

In [11]:
# Test data
v1 = strs2u8mat(["AA", "BB", "CC"])
v2 = strs2u8mat(["X", "Y", "Z", "W"])
v3 = strs2u8mat(["11", "22"])
pipe = sep(b"|")

print(f"v1 shape: {v1.shape}")
print(f"v2 shape: {v2.shape}")
print(f"v3 shape: {v3.shape}")

v1 shape: (3, 2)
v2 shape: (4, 1)
v3 shape: (2, 2)


In [12]:
# 2-way cartesian product
cp2 = cartesian_product((v1, v2))
print(f"cartesian_product 2-way: {v1.shape} x {v2.shape} -> {cp2.shape}")
for s in u8m2S(cp2).astype(f"U{cp2.shape[1]}"):
    print(f"  {s}")

cartesian_product 2-way: (3, 2) x (4, 1) -> (12, 3)
  AAX
  AAY
  AAZ
  AAW
  BBX
  BBY
  BBZ
  BBW
  CCX
  CCY
  CCZ
  CCW


In [13]:
# With separator
cp2s = cartesian_product((v1, pipe, v2))
print(f"cartesian_product with sep: {v1.shape} x {v2.shape} -> {cp2s.shape}")
for s in u8m2S(cp2s).astype(f"U{cp2s.shape[1]}"):
    print(f"  {s}")

cartesian_product with sep: (3, 2) x (4, 1) -> (12, 4)
  AA|X
  AA|Y
  AA|Z
  AA|W
  BB|X
  BB|Y
  BB|Z
  BB|W
  CC|X
  CC|Y
  CC|Z
  CC|W


In [14]:
# 3-way cartesian product
cp3 = cartesian_product((v1, v2, v3))
print(f"cartesian_product 3-way: {v1.shape} x {v2.shape} x {v3.shape} -> {cp3.shape}")
for s in u8m2S(cp3).astype(f"U{cp3.shape[1]}")[:6]:
    print(f"  {s}")
print("  ...")

cartesian_product 3-way: (3, 2) x (4, 1) x (2, 2) -> (24, 5)
  AAX11
  AAX22
  AAY11
  AAY22
  AAZ11
  AAZ22
  ...


In [15]:
# 3-way with separators
cp3s = cartesian_product((v1, pipe, v2, pipe, v3))
print(f"cartesian_product 3-way with seps: -> {cp3s.shape}")
for s in u8m2S(cp3s).astype(f"U{cp3s.shape[1]}")[:6]:
    print(f"  {s}")
print("  ...")

cartesian_product 3-way with seps: -> (24, 7)
  AA|X|11
  AA|X|22
  AA|Y|11
  AA|Y|22
  AA|Z|11
  AA|Z|22
  ...


### Cartesian Product Benchmarks

In [16]:
# Realistic sizes for asset-straddle generation
assets = [f"Asset{i:03d} Comdty" for i in range(200)]
yearmonths = [f"{y}-{m:02d}" for y in range(2001, 2026) for m in range(1, 13)]

v_assets = strs2u8mat(assets)
v_ym = strs2u8mat(yearmonths)

print(f"Assets: {len(assets)}, width={v_assets.shape[1]}")
print(f"Yearmonths: {len(yearmonths)}, width={v_ym.shape[1]}")
print(f"Output rows: {len(assets) * len(yearmonths):,}")

Assets: 200, width=15
Yearmonths: 300, width=7
Output rows: 60,000


In [17]:
pipe = sep(b"|")

# Warm up JIT
_ = cartesian_product((v_assets, v_ym))
_ = cartesian_product((v_assets, pipe, v_ym))
_ = cartesian_product_np((v_assets, v_ym))

def bench_cp2():
    return cartesian_product((v_assets, v_ym))

def bench_cp2_sep():
    return cartesian_product((v_assets, pipe, v_ym))

def bench_cp2_np():
    return cartesian_product_np((v_assets, v_ym))

def bench_python_product():
    import itertools
    return [a + ym for a, ym in itertools.product(assets, yearmonths)]

print(f"cartesian_product (Numba):   {run_bench(bench_cp2):9.2f} ms")
print(f"cartesian_product with sep:  {run_bench(bench_cp2_sep):9.2f} ms")
print(f"cartesian_product_np:        {run_bench(bench_cp2_np):9.2f} ms")
print(f"Python itertools:            {run_bench(bench_python_product):9.2f} ms")

cartesian_product (Numba):        0.27 ms
cartesian_product with sep:       0.34 ms
cartesian_product_np:             1.69 ms
Python itertools:                 2.29 ms


In [18]:
# Three-way product benchmark
v_ym2 = v_ym  # expiry yearmonths = same as entry for test
print(f"Three-way product: {len(assets)} x {len(yearmonths)} x {len(yearmonths)} = {len(assets)*len(yearmonths)*len(yearmonths):,} rows")

# Warm up
_ = cartesian_product((v_assets, v_ym, v_ym2))
_ = cartesian_product_np((v_assets, v_ym, v_ym2))

def bench_cp3():
    return cartesian_product((v_assets, v_ym, v_ym2))

def bench_cp3_par():
    return cartesian_product_par((v_assets, v_ym, v_ym2))

def bench_cp3_np():
    return cartesian_product_np((v_assets, v_ym, v_ym2))

def bench_python_product3():
    import itertools
    return [a + e + x for a, e, x in itertools.product(assets, yearmonths, yearmonths)]

print(f"cartesian_product     (Numba):   {run_bench(bench_cp3):9.2f} ms")
print(f"cartesian_product_par (Numba):   {run_bench(bench_cp3_par):9.2f} ms")
print(f"cartesian_product_np:            {run_bench(bench_cp3_np):9.2f} ms")
print(f"Python itertools:                {run_bench(bench_python_product3):9.2f} ms")

Three-way product: 200 x 300 x 300 = 18,000,000 rows
cartesian_product     (Numba):      154.23 ms
cartesian_product_par (Numba):       35.49 ms
cartesian_product_np:               792.60 ms
Python itertools:                  1346.57 ms


### Parallel Cartesian Product

Compare sequential odometer vs parallel stride-based implementations.

In [19]:
# Warm up parallel JIT
_ = cartesian_product_par((v_assets, v_ym))

def bench_cp3_par():
    return cartesian_product_par((v_assets, v_ym, v_ym))

print(f"Two-way product: {len(assets)} x {len(yearmonths)} = {len(assets)*len(yearmonths):,} rows")
print(f"cartesian_product (seq):     {run_bench(bench_cp3):9.2f} ms")
print(f"cartesian_product_par:       {run_bench(bench_cp3_par):9.2f} ms")

Two-way product: 200 x 300 = 60,000 rows
cartesian_product (seq):        159.98 ms
cartesian_product_par:           34.22 ms


In [20]:
# Three-way: parallel should shine here
_ = cartesian_product_par((v_assets, v_ym, v_ym2))

def bench_cp3_par():
    return cartesian_product_par((v_assets, v_ym, v_ym2))

print(f"Three-way product: {len(assets)} x {len(yearmonths)} x {len(yearmonths)} = {len(assets)*len(yearmonths)*len(yearmonths):,} rows")
print(f"cartesian_product (seq):     {run_bench(bench_cp3):9.2f} ms")
print(f"cartesian_product_par:       {run_bench(bench_cp3_par):9.2f} ms")

Three-way product: 200 x 300 x 300 = 18,000,000 rows
cartesian_product (seq):        159.34 ms
cartesian_product_par:           33.51 ms


In [21]:
# Scaling comparison: vary output size
print("Sequential vs Parallel scaling:")
print(f"{'Rows':>12} {'seq (ms)':>10} {'par (ms)':>10} {'speedup':>8}")
print("-" * 44)

for n_assets in [1, 10, 100, 1000 ]:
    test_assets = strs2u8mat([f"Asset{i:03d}" for i in range(n_assets)])
    test_ym = strs2u8mat([f"{y}-{m:02d}" for y in range(2001, 2025) for m in range(1, 13)])
    total_rows = n_assets * test_ym.shape[0] * test_ym.shape[0]
    
    # Warm up
    _ = cartesian_product((test_assets, test_ym))
    _ = cartesian_product_par((test_assets, test_ym))
    
    t_seq = run_bench(lambda: cartesian_product((test_assets, test_ym, test_ym)))
    t_par = run_bench(lambda: cartesian_product_par((test_assets, test_ym, test_ym)))
    speedup = t_seq / t_par if t_par > 0 else float('inf')
    
    print(f"{total_rows:>12,} {t_seq:>10.2f} {t_par:>10.2f} {speedup:>7.2f}x")

Sequential vs Parallel scaling:
        Rows   seq (ms)   par (ms)  speedup
--------------------------------------------
      82,944       0.50       0.32    1.57x
     829,440       4.92       1.17    4.19x
   8,294,400      59.16      13.82    4.28x
  82,944,000     629.70     136.55    4.61x


## Unfurl Operations

Expand matrices by duplicating rows according to a specification.

In [22]:
# Simple unfurl: duplicate rows by count
mat = strs2u8mat(["AA", "BB", "CC"])
counts = np.array([2, 1, 3], dtype=np.uint8)

out, sidx = unfurl(mat, counts)
print(f"unfurl: mat {mat.shape} with counts {counts.tolist()} -> {out.shape}")
print(f"  src_idx: {sidx.tolist()}")
for s in u8m2S(out).astype(f"U{out.shape[1]}"):
    print(f"  {s}")

unfurl: mat (3, 2) with counts [2, 1, 3] -> (6, 2)
  src_idx: [0, 0, 1, 2, 2, 2]
  AA
  AA
  BB
  CC
  CC
  CC


In [23]:
# unfurl_by_spec with single-byte data items
print("unfurl_by_spec (1-byte items):")
mat2 = strs2u8mat(["STR1", "STR2"])
# spec: [count, item0, item1, item2, item3]
# Row 0: 2 expansions with data N, F
# Row 1: 4 expansions with data N, N, F, F
spec = np.array([
    [2, ord('N'), ord('F'), 0, 0],
    [4, ord('N'), ord('N'), ord('F'), ord('F')],
], dtype=np.uint8)

out2, sidx2 = unfurl_by_spec(mat2, spec)
print(f"  mat {mat2.shape}, spec {spec.shape} -> out {out2.shape}")
print(f"  src_idx: {sidx2.tolist()}")
for s in u8m2S(out2).astype(f"U{out2.shape[1]}"):
    print(f"  {s}")

unfurl_by_spec (1-byte items):
  mat (2, 4), spec (2, 5) -> out (6, 5)
  src_idx: [0, 0, 1, 1, 1, 1]
  STR1N
  STR1F
  STR2N
  STR2N
  STR2F
  STR2F


In [24]:
# unfurl_by_spec with 2-byte data items
print("unfurl_by_spec (2-byte items):")
mat3 = strs2u8mat(["AA", "BB"])
# spec: [count, item0_b0, item0_b1, item1_b0, item1_b1, ...]
# Row 0: 2 expansions with data "NN", "FF"
# Row 1: 3 expansions with data "XX", "YY", "ZZ"
spec2 = np.array([
    [2, ord('N'), ord('N'), ord('F'), ord('F'), 0, 0],
    [3, ord('X'), ord('X'), ord('Y'), ord('Y'), ord('Z'), ord('Z')],
], dtype=np.uint8)

out3, sidx3 = unfurl_by_spec(mat3, spec2)
print(f"  mat {mat3.shape}, spec {spec2.shape} -> out {out3.shape}")
print(f"  src_idx: {sidx3.tolist()}")
for s in u8m2S(out3).astype(f"U{out3.shape[1]}"):
    print(f"  {s}")

unfurl_by_spec (2-byte items):
  mat (2, 2), spec (2, 7) -> out (5, 4)
  src_idx: [0, 0, 1, 1, 1]
  AANN
  AAFF
  BBXX
  BBYY
  BBZZ


In [25]:
# unfurl_by_spec_sep
print("unfurl_by_spec_sep:")
ASCII_PIPE = np.uint8(ord("|"))
out4, sidx4 = unfurl_by_spec_sep(mat2, spec, ASCII_PIPE)
print(f"  mat {mat2.shape}, spec {spec.shape}, sep='|' -> out {out4.shape}")
for s in u8m2S(out4).astype(f"U{out4.shape[1]}"):
    print(f"  {s}")

unfurl_by_spec_sep:
  mat (2, 4), spec (2, 5), sep='|' -> out (6, 6)
  STR1|N
  STR1|F
  STR2|N
  STR2|N
  STR2|F
  STR2|F


In [26]:
# unfurl_concat with calendar output
print("unfurl_concat with calendar:")
assets_mat = strs2u8mat(["CL", "GC"])
ranges = np.array([
    [2024, 1, 2024, 1],  # CL: Jan 2024 (31 days)
    [2024, 2, 2024, 2],  # GC: Feb 2024 (29 days, leap year)
], dtype=np.int64)
cal_idx, cal_dates = make_calendar_from_ranges(ranges)
result = unfurl_concat(assets_mat, cal_dates, cal_idx)
print(f"  assets {assets_mat.shape}, dates {cal_dates.shape} -> {result.shape}")
print(f"  First 3 rows:")
for s in u8m2S(result[:3]).astype(f"U{result.shape[1]}"):
    print(f"    {s}")
print(f"  Last 3 rows:")
for s in u8m2S(result[-3:]).astype(f"U{result.shape[1]}"):
    print(f"    {s}")

unfurl_concat with calendar:
  assets (2, 2), dates (60, 10) -> (60, 12)
  First 3 rows:
    CL2024-01-01
    CL2024-01-02
    CL2024-01-03
  Last 3 rows:
    GC2024-02-27
    GC2024-02-28
    GC2024-02-29


In [27]:
# unfurl_concat_sep
print("unfurl_concat_sep:")
result_sep = unfurl_concat_sep(assets_mat, cal_dates, cal_idx, ASCII_PIPE)
print(f"  -> {result_sep.shape}")
print(f"  First 3:")
for s in u8m2S(result_sep[:3]).astype(f"U{result_sep.shape[1]}"):
    print(f"    {s}")

unfurl_concat_sep:
  -> (60, 13)
  First 3:
    CL|2024-01-01
    CL|2024-01-02
    CL|2024-01-03


### Unfurl Benchmarks

In [28]:
# Setup: 1000 rows, variable expansion counts
n_rows = 1000
bench_mat = strs2u8mat([f"Row{i:04d}" for i in range(n_rows)])
bench_counts = np.array([((i % 10) + 1) for i in range(n_rows)], dtype=np.uint8)
total_expanded = int(bench_counts.sum())

print(f"Input: {n_rows} rows, total expanded: {total_expanded:,}")

# Warm up
_ = unfurl(bench_mat, bench_counts)

def bench_unfurl():
    return unfurl(bench_mat, bench_counts)

def bench_python_unfurl():
    result = []
    for i, row in enumerate(bench_mat):
        for _ in range(bench_counts[i]):
            result.append(row.copy())
    return result

print(f"unfurl (Numba):  {run_bench(bench_unfurl):9.2f} ms")
print(f"Python loop:     {run_bench(bench_python_unfurl):9.2f} ms")

Input: 1000 rows, total expanded: 5,500
unfurl (Numba):       0.02 ms
Python loop:          1.11 ms


In [29]:
# Benchmark unfurl_by_spec
bench_spec = np.zeros((n_rows, 1 + 10), dtype=np.uint8)  # max 10 expansions
for i in range(n_rows):
    c = (i % 10) + 1
    bench_spec[i, 0] = c
    for j in range(c):
        bench_spec[i, 1 + j] = ord('A') + (j % 26)

_ = unfurl_by_spec(bench_mat, bench_spec)

def bench_unfurl_spec():
    return unfurl_by_spec(bench_mat, bench_spec)

print(f"unfurl_by_spec:  {run_bench(bench_unfurl_spec):9.2f} ms")

unfurl_by_spec:       0.02 ms


## Date Parsing

Parse year/month/day from uint8-encoded ASCII date strings.

In [30]:
# Generate test data: 25 years of dates using make_calendar_from_ranges
date_ranges = np.array([[2001, 1, 2025, 12]], dtype=np.int64)
_, date_cal = make_calendar_from_ranges(date_ranges)

# Also create yearmonth data (7 cols) by slicing the full dates
ym_cal = date_cal[:, :7]  # Just "YYYY-MM"

print(f"Full dates (YYYY-MM-DD): {date_cal.shape[0]:,} rows, {date_cal.shape[1]} cols")
print(f"Year-months (YYYY-MM):   {ym_cal.shape[0]:,} rows, {ym_cal.shape[1]} cols")
print(f"\nSample dates:")
for s in u8m2S(date_cal)[:5].astype(f"U{date_cal.shape[1]}"):
    print(f"  {s}")

Full dates (YYYY-MM-DD): 9,131 rows, 10 cols
Year-months (YYYY-MM):   9,131 rows, 7 cols

Sample dates:
  2001-01-01
  2001-01-02
  2001-01-03
  2001-01-04
  2001-01-05


In [31]:
# Verify parsing works correctly
print("get_uint8_ymd examples:")
for i in [0, 100, 1000]:
    y, m, d = get_uint8_ymd(date_cal[i])
    s = u8m2S(date_cal[i:i+1])[0].decode()
    print(f"  {s} -> ({y}, {m}, {d})")

print("\nget_uint8_ym examples:")
for i in [0, 100, 1000]:
    y, m = get_uint8_ym(ym_cal[i])
    s = u8m2S(ym_cal[i:i+1])[0].decode()
    print(f"  {s} -> ({y}, {m})")

get_uint8_ymd examples:
  2001-01-01 -> (2001, 1, 1)
  2001-04-11 -> (2001, 4, 11)
  2003-09-28 -> (2003, 9, 28)

get_uint8_ym examples:
  2001-01 -> (2001, 1)
  2001-04 -> (2001, 4)
  2003-09 -> (2003, 9)


In [32]:
# Benchmark: parse all dates
from numba import njit

@njit
def bench_parse_ymd(cal):
    """Parse all rows, return sum of days (to prevent dead code elimination)."""
    total = 0
    for i in range(cal.shape[0]):
        y, m, d = get_uint8_ymd(cal[i])
        total += d
    return total

@njit
def bench_parse_ym(cal):
    """Parse all rows, return sum of months."""
    total = 0
    for i in range(cal.shape[0]):
        y, m = get_uint8_ym(cal[i])
        total += m
    return total

# Warm up JIT
_ = bench_parse_ymd(date_cal)
_ = bench_parse_ym(ym_cal)

n_rows = date_cal.shape[0]
t_ymd = run_bench(lambda: bench_parse_ymd(date_cal))
t_ym = run_bench(lambda: bench_parse_ym(ym_cal))

print(f"Parsing {n_rows:,} dates:")
print(f"  get_uint8_ymd: {t_ymd:.2f} ms  ({n_rows/t_ymd/1000:.1f}M rows/sec)")
print(f"  get_uint8_ym:  {t_ym:.2f} ms  ({n_rows/t_ym/1000:.1f}M rows/sec)")

Parsing 9,131 dates:
  get_uint8_ymd: 0.01 ms  (713.8M rows/sec)
  get_uint8_ym:  0.01 ms  (779.9M rows/sec)


In [33]:
# Compare with Python string parsing
import datetime

def python_parse_ymd(cal):
    """Python equivalent: decode bytes and parse."""
    total = 0
    for i in range(cal.shape[0]):
        s = bytes(cal[i]).decode('ascii')
        dt = datetime.datetime.strptime(s, "%Y-%m-%d")
        total += dt.day
    return total

def python_parse_ym(cal):
    """Python equivalent for year-month."""
    total = 0
    for i in range(cal.shape[0]):
        s = bytes(cal[i]).decode('ascii')
        # Parse YYYY-MM
        total += int(s[5:7])
    return total

# Use smaller subset for Python (it's slow)
n_subset = 10000
t_py_ymd = run_bench(lambda: python_parse_ymd(date_cal[:n_subset]))
t_py_ym = run_bench(lambda: python_parse_ym(ym_cal[:n_subset]))

# Scale up for comparison
t_py_ymd_scaled = t_py_ymd * (n_rows / n_subset)
t_py_ym_scaled = t_py_ym * (n_rows / n_subset)

print(f"Python parsing (extrapolated to {n_rows:,} rows):")
print(f"  strptime YYYY-MM-DD: {t_py_ymd_scaled:.0f} ms  ({n_rows/t_py_ymd_scaled/1000:.2f}M rows/sec)")
print(f"  string slice YYYY-MM: {t_py_ym_scaled:.0f} ms  ({n_rows/t_py_ym_scaled/1000:.2f}M rows/sec)")
print(f"\nSpeedup vs Python:")
print(f"  get_uint8_ymd: {t_py_ymd_scaled/t_ymd:.0f}x faster")
print(f"  get_uint8_ym:  {t_py_ym_scaled/t_ym:.0f}x faster")

Python parsing (extrapolated to 9,131 rows):
  strptime YYYY-MM-DD: 22 ms  (0.41M rows/sec)
  string slice YYYY-MM: 3 ms  (3.47M rows/sec)

Speedup vs Python:
  get_uint8_ymd: 1723x faster
  get_uint8_ym:  225x faster


In [35]:

x=make_ym_matrix((2025,1,2025,12))
print(u8m2s(x))
s1=sep(b"|")
s2=sep(b"|"+b" "*7+b"|")
specs = cartesian_product((s1,x,s2))
print(u8m2s(specs)[0:3,])
add_months2specs_inplace(specs[:,9:16],specs[:,1:7],[i%2+1 for i in range(specs.shape[0])])
u8m2s(specs)[0:3,]


['2025-01' '2025-02' '2025-03' '2025-04' '2025-05' '2025-06' '2025-07'
 '2025-08' '2025-09' '2025-10' '2025-11' '2025-12']
['|2025-01|       |' '|2025-02|       |' '|2025-03|       |']


array(['|2025-01|2025-02|', '|2025-02|2025-04|', '|2025-03|2025-04|'],
      dtype=StringDType())

In [19]:
from specparser.amt import loader,schedules

AMT_PATH = "../data/amt.yml"

def straddles(amt):
    assets = [x for [x] in loader.assets(amt)["rows"]]
    assets_u8m = strs2u8mat(assets)
    straddles=[]
    for asset in assets:
        shed = schedules.get_schedule(amt,asset)
        cols = shed["columns"]
        ntrcs, ntrvs = [], []
        xprcs, xprvs = [], []
        wgts=[]
        for row in shed["rows"]:
            rd=dict(zip(cols,row))
            ntrcs.append(rd["ntrc"])
            ntrvs.append(f"{int(rd["ntrv"]):02d}")
            xprcs.append(rd["xprc"])
            xprvs.append(rd["xprv"])
            wgts.append(f"{float(rd["wgt"]):0.1f}")
        ntrcs_u8m=strs2u8mat(ntrcs)
        ntrvs_u8m=strs2u8mat(ntrvs)
        xprcs_u8m=strs2u8mat(xprcs,8)
        xprvs_u8m=strs2u8mat(xprvs,2)
        wgts_u8m=strs2u8mat(wgts,5)
        asset_u8m=strs2u8mat([asset]*len(ntrcs),width=assets_u8m.shape[1])
        spec = np.concatenate([
            sep(b"|",ntrcs_u8m.shape[0]),
            asset_u8m,
            sep(b"|"+b" "*7+b"|"+b" "*7+b"|",ntrcs_u8m.shape[0]),
            ntrcs_u8m,
            sep(b"|",ntrcs_u8m.shape[0]),
            ntrvs_u8m,
            sep(b"|",ntrcs_u8m.shape[0]),
            xprcs_u8m,
            sep(b"|",ntrcs_u8m.shape[0]),
            xprvs_u8m,
            sep(b"|",ntrcs_u8m.shape[0]),
            wgts_u8m,
            sep(b"|",ntrcs_u8m.shape[0])
        ],axis=1)
        straddles.append(spec)
        #print(u8m2s(spec))
    return np.vstack(straddles)

def nth_occurrence(x, v, n):
    # n is 1-based: n=1 returns the first occurrence
    idx = np.flatnonzero(x == v)
    if idx.size < n:
        return -1  # or raise
    return int(idx[n - 1])

straddles_u8m = straddles(AMT_PATH)
all_straddles = u8m2s(straddles_u8m)
#print(f"{run_bench(straddles):0.2f} msec")
yrs = [f"{y:4d}" for y in range(2001,2028)]
yrs_u8m = strs2u8mat(yrs)
months = [f"{m:02d}" for m in range(1,12)]
months_u8m = strs2u8mat(months)
straddles_ym_u8m = cartesian_product((straddles_u8m,yrs_u8m,sep(b"-"),months_u8m,sep(b"|")))
straddles_ym = u8m2s(straddles_ym_u8m)
print(straddles_ym.shape)
tenor_idx=nth_occurrence(straddles_ym_u8m[0,:],ord("|"),4)
tenor=straddles_ym_u8m[:,tenor_idx+1:tenor_idx+2]
source_idx=nth_occurrence(straddles_ym_u8m[0,:],ord("|"),9)
source=straddles_ym_u8m[:,source_idx+1:source_idx+7]
target_idx=nth_occurrence(straddles_ym_u8m[0,:],ord("|"),3)
target=straddles_ym_u8m[:,target_idx+1:target_idx+7]
copyto_idx=nth_occurrence(straddles_ym_u8m[0,:],ord("|"),2)
copyto=straddles_ym_u8m[:,copyto_idx+1:copyto_idx+7]
add_months2specs_inplace_NF(target,source,tenor)
add_months2specs_inplace(copyto,source,np.zeros(tenor.shape[0],dtype=np.uint8))
straddles_ym1 = u8m2s(straddles_ym_u8m[:,0:source_idx+1])
print(straddles_ym1[np.random.randint(0, straddles_ym1.shape[0], size=10)])




(276210,)
['|AUDJPY Curncy   |2022-05|2022-06|N|00|BD      |5 |25.0 |'
 '|EURINR Curncy   |2019-01|2019-02|N|00|BD      |13|25.0 |'
 '|TY Comdty       |2003-01|2003-02|N|05|OVERRIDE|  |33.3 |'
 '|ABNB US Equity  |2021-05|2021-07|F|10|F       |3 |12.5 |'
 '|LN Comdty       |2023-06|2023-07|N|00|OVERRIDE|  |33.3 |'
 '|SPY US Equity   |2014-07|2014-08|N|00|F       |1 |25.0 |'
 '|PANW US Equity  |2006-01|2006-02|N|05|F       |3 |33.3 |'
 '|V US Equity     |2011-06|2011-07|N|00|F       |3 |33.3 |'
 '|MBG GR Equity   |2022-08|2022-10|F|10|F       |3 |12.5 |'
 '|NOKJPY Curncy   |2009-01|2009-02|N|00|BD      |4 |25.0 |']
